In [ ]:
import os
import json
import pytz
from datetime import datetime, timedelta
import statsmodels.api as statsmodel
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
### DEFINE SOME USEFUL FUNCTIONS 



def load_json_obj(file_name):
    tweets = []   # define a list to store json object
    f = open(file_name, 'r', encoding = 'utf-8')
    data_in_row = f.readline()    # read a row (json string)
    while len(data_in_row)!=0:    # if this row is nor empty,begin the loop
        tweet = json.loads(data_in_row)    # transfer json string to json object
        tweets.append(tweet)    # store json object into the list
        line = f.readline()     # read a row
    f.close()
    return tweets    



def calc_statistics(file_name):
    
    # load the json object list
    tweets = load_json_obj(file_name)
    
    # define lists to store information
    num_tweets_list = []   # store the number of tweets (each tweet is represented by 1)
    num_retweets_list = []  # store the number of retweets for each tweet
    num_users_list = []   # store the number of users (each user is represented by 1)
    num_followers_list = []   # store the number of followers for each user
        
    # use start time and end time to calculate of the number total hours
    start_time = tweets[0]['citation_date']
    end_time = tweets[-1]['citation_date']
    num_hours = math.ceil((end_time - start_time)/3600)   # math,ceil(15.0)=15, math.ceil(15.2)=16
    
    # find the values in the lists
    for tweet in tweets: 
        num_tweets_list.append(1)
        num_retweets_list.append(tweet['metrics']['citations']['total'])
        num_users_list.append(1) 
        num_followers_list.append(tweet['author']['followers'])   # the number of followers for the user
        
    # calculate statistics
    num_tweets = len(num_tweets_list)
    num_retweets = np.sum(num_retweets_list)
    num_users = len(user_id_list)
    num_followers = np.sum(num_followers_list)
    avg_tweets_per_hour = num_tweets / num_hours
    avg_followers_per_tweet = num_followers / num_tweets
    avg_retweets_per_tweet = num_retweets / num_tweets
    
    print('For %s :' %flie_name)
    print('Average number of tweets per hour is ',avg_tweets_per_hour)
    print('Average number of followers of users posting the tweets per tweet is 'avg_followers_per_tweet)
    print('Average number of retweets per tweet is 'avg_retweets_per_tweet)
    return avg_tweets_per_hour, avg_followers_per_tweet, avg_retweets_per_tweet




def plot_num_tweets_per_hour(flie_name, hashtag_name):
    
    # load the json object list
    tweets = load_json_obj(file_name)
    
    # use start time and end time to calculate of the number total hours
    start_time = tweets[0]['citation_date']
    end_time = tweets[-1]['citation_date']
    num_hours = math.ceil((end_time - start_time)/3600)   # math,ceil(15.0)=15, math.ceil(15.2)=16
    
    # calculate the number of tweets in each hour
    num_tweets_in_hours = np.zeros(num_hours).tolist()
    for i in range(len(tweets)):
        hour_idx = int((tweets[i]['citation_date'] - start_time)/3600)   # in which hour
        num_tweets_in_hours[hour_idx] += 1
    
    # plot the figure
    plt.figure()
    plt.bar(range(num_hours), num_tweets_in_hours, width=1)
    plt.xlabel('Hour')
    plt.ylabel('Number of tweets')
    plt.title(('Number of tweets in hour for ' + hashtag_name))
    plt.show()
    

"""    
def extract_five_features(file_name):
    
    # load the json object list
    tweets = load_json_obj(file_name)
    
    # define a dictionary to store feature values for each hour
    hourly_features = {}  
    
    for tweet in tweets:
        # deal with time format
        tweet_time_UNIX = tweet['citation_date'] 
        pst_tz = pytz.timezone('America/Los_Angeles')
        tweet_time = datetime.datetime.fromtimestamp(tweet_time_UNIX,pst_tz) 
        tweet_hour_time = datetime.datetime(tweet_time.year, tweet_time.month, tweet_time.day, tweet_time.hour, 0, 0)
        tweet_hour_time = str(tweet_hour_time)
        # calculate feature values    
        if tweet_hour_time not in hourly_features: 
            hourly_features[tweet_hour_time] = {'num_tweets':0, 'num_retweets':0, 'num_followers':0, 'max_followers':0, 'hour':0}
        hourly_features[tweet_hour_time]['num_tweets'] += 1
        hourly_features[tweet_hour_time]['num_retweets'] += tweet['metrics']['citations']['total']
        hourly_features[tweet_hour_time]['num_followers'] += tweet['author']['followers']
        if tweet['author']['followers'] > hourly_features[tweet_hour_time]['max_followers']:
            hourly_features[tweet_hour_time]['max_followers'] = tweet['author']['followers']
        hourly_features[tweet_hour_time]['hour'] = tweet_time.hour
                 
    return hourly_features

"""

def extract_five_features(flie_name):
    
    # load the json object list
    tweets = load_json_obj(file_name)
    
    # use start time and end time to calculate of the number total hours
    start_time = tweets[0]['citation_date']
    end_time = tweets[-1]['citation_date']
    num_hours = math.ceil((end_time - start_time)/3600)   # math,ceil(15.0)=15, math.ceil(15.2)=16
    
    # define a np.array to store feature values
    hourly_features = np.zeros([num_hours,5])
    
    # calculate the feature values in each hour
    num_tweets_in_hours = np.zeros(num_hours).tolist()
    for tweet in tweets:
        hour_idx = int((tweet['citation_date'] - start_time)/3600)   # in which hour (in which index)
        hourly_features[hour_idx][0] += 1  # total number of tweets
        hourly_features[hour_idx][1] += tweet['metrics']['citations']['total']  # total number of retweets
        hourly_features[hour_idx][2] += tweet['author']['followers']  # total number of followers
        if tweet['author']['followers'] > hourly_features[hour_idx][3]:
            hourly_features[hour_idx][3] += tweet['author']['followers']  # maximum followers
        tweet_time_UNIX = tweet['citation_date'] 
        pst_tz = pytz.timezone('America/Los_Angeles')
        tweet_time = datetime.datetime.fromtimestamp(tweet_time_UNIX,pst_tz) 
        hourly_features[hour_idx][4] = tweet_time.hour  # hour
        
    return hourly_features



def extract_more_features(flie_name):
    
    # load the json object list
    tweets = load_json_obj(file_name)
    
    # use start time and end time to calculate of the number total hours
    start_time = tweets[0]['citation_date']
    end_time = tweets[-1]['citation_date']
    num_hours = math.ceil((end_time - start_time)/3600)   # math,ceil(15.0)=15, math.ceil(15.2)=16
    
    # define a np.array to store feature values
    hourly_features = np.zeros([num_hours,14])
    
    # calculate the feature values in each hour
    num_tweets_in_hours = np.zeros(num_hours).tolist()
    for tweet in tweets:
        hour_idx = int((tweet['citation_date'] - start_time)/3600)   # in which hour (in which index)
        hourly_features[hour_idx][0] += 1  # total number of tweets
        hourly_features[hour_idx][1] += tweet['metrics']['citations']['total']  # total number of retweets
        hourly_features[hour_idx][2] += tweet['author']['followers']  # total number of followers
        if tweet['author']['followers'] > hourly_features[hour_idx][3]:
            hourly_features[hour_idx][3] += tweet['author']['followers']  # maximum followers
        tweet_time_UNIX = tweet['citation_date'] 
        pst_tz = pytz.timezone('America/Los_Angeles')
        tweet_time = datetime.datetime.fromtimestamp(tweet_time_UNIX,pst_tz) 
        hourly_features[hour_idx][4] = tweet_time.hour  # hour
        """ #user_id needs to be defined
        hourly_features[hour_idx][5] += tweet['metrics']['impressions']
        hourly_features[hour_idx][6] += tweet['metrics']['momentum']
        hourly_features[hour_idx][7] += tweet['tweet']['favorite_count']
        hourly_features[hour_idx][8] += tweet['metrics']['ranking_score']
        hourly_features[hour_idx][9] += tweet['metrics']['acceleration']
        hourly_features[hour_idx][10] += tweet['metrics']['citations']['replies']
        if tweet['tweet']['user']['id'] not in dict_unique_users[hour_num]:
            dict_unique_users[hour_num].add(tweet['tweet']['user']['id'])
            hourly_features[hour_idx][11] += 1
        if tweet['author']['name'] not in dict_unique_authors[hour_num]:
            dict_unique_authors[hour_num].add(tweet['author']['name'])
            hourly_features[hour_idx][12] += 1
        hourly_features[hour_idx][13] += len(tweet['tweet']['entities']['user_mentions'])
        """
    return hourly_features

In [ ]:
### QUESTION 1


file_names = ['tweets_#gohawks.txt', 'tweets_#gopatriots.txt', 'tweets_#nfl.txt',
              'tweets_#patriots.txt', 'tweets_#sb49.txt', 'tweets_#superbowl.txt']
for file_name in file_names:
    calc_statistics(file_name)
    

In [ ]:
### QUESTION 2


plot_num_tweets_per_hour('tweets_#superbowl.txt', '#SuperBowl')
plot_num_tweets_per_hour('tweets_#nfl.txt', '#NFL')


In [ ]:
### QUESTION 3


file_names = ['tweets_#gohawks.txt', 'tweets_#gopatriots.txt', 'tweets_#nfl.txt',
              'tweets_#patriots.txt', 'tweets_#sb49.txt', 'tweets_#superbowl.txt']

for file_name in file_names:
    # X_train & y
    X = extract_five_features(flie_name)
    X_train = X[:-1,:]  
    y = X[1:,0]
    # train a model
    RegressionModel = statsmodel.OLS(y, X_train).fit()
    y_pred = RegressionModel.predict()
    # do the calculation
    print('*'*50)
    print('For ', file_name,':')
    mse = metrics.mean_squared_error(y, y_pred)
    print('MSE value: ',mse)
    print('Summary report: ')
    print(model.summary())
    print('\n\n')
    
    

In [ ]:
### QUESTION 4


file_names = ['tweets_#gohawks.txt', 'tweets_#gopatriots.txt', 'tweets_#nfl.txt',
              'tweets_#patriots.txt', 'tweets_#sb49.txt', 'tweets_#superbowl.txt']

for file_name in file_names:
    # X_train & y
    X = extract_more_features(flie_name)
    X_train = X[:-1,:]  
    y = X[1:,0]
    # train a model
    RegressionModel = statsmodel.OLS(y, X_train).fit()
    y_pred = RegressionModel.predict()
    # do the calculation
    print('*'*50)
    print('For ', file_name,':')
    mse = metrics.mean_squared_error(y, y_pred)
    print('MSE value: ',mse)
    print('\n\n')
    
    

In [ ]:
### QUESTION 5